In [ ]:
import pandas as pd
import numpy as np
import jieba

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, cross_validate, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB  
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier 
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics

from io import StringIO 
col = ['Column2', 'Column3'] 
df = df[col] 
df = df[pd.notnull(df['Column3'])] 
df.columns = ['Column2', 'Column3'] 
df['category_id'] = df['Column2'].factorize()[0] 
category_id_df = df[['Column2', 'category_id']].drop_duplicates().sort_values('category_id') 
category_to_id = dict(category_id_df.values) 
id_to_category = dict(category_id_df[['category_id', 'Column2']].values) 
df.head()

In [ ]:
import pandas as pd
import csv

Class=list()
intr=list()
with open('NLP_HW02.csv', 'r', newline='', encoding='utf-8') as csvfile:
    rows = csv.DictReader(csvfile)
    for row in rows:
      row['Column2'] = row['Column2'].split(' ')[0]
      Class.append(row['Column2'])
      intr.append(row['Column3'])

for i in range(6001):
  with open('NLP_HW02_copy.csv', 'a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([  Class[i], intr[i]])

In [ ]:
from io import StringIO 
df = pd.read_csv('NLP_HW02_copy.csv')
col = ['分類', '劇情介紹'] 
df = df[col] 
df = df[pd.notnull(df['劇情介紹'])] 
df.columns = ['分類', '劇情介紹'] 
df['category_id'] = df['分類'].factorize()[0] 
category_id_df = df[['分類', 'category_id']].drop_duplicates().sort_values('category_id') 
category_to_id = dict(category_id_df.values) 
id_to_category = dict(category_id_df[['category_id', '分類']].values) 
df.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english') 
features = tfidf.fit_transform(df.劇情介紹).toarray() 
labels = df.category_id 
features.shape


In [ ]:
from sklearn.feature_selection import chi2 
import numpy as np 
N = 2 
for Product, category_id in sorted(category_to_id.items()): 
  features_chi2 = chi2(features, labels == category_id) 
  indices = np.argsort(features_chi2[0]) 
  feature_names = np.array(tfidf.get_feature_names())[indices] 
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1] 
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2] 
  print("# '{}':".format(Product)) 
  print(" . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:]))) 
  print(" . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))


In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['劇情介紹'], df['分類'], test_size=0.9, random_state = 0)
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(X_train) 
tfidf_transformer = TfidfTransformer() 
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts) 
clf = MultinomialNB().fit(X_train_tfidf, y_train)

print(clf.predict(count_vect.transform(["2010/12/03 早場搶先上映！★2010金馬影展開幕片★2010東京影展  2010釜山影展★台灣三大實力導演聯合執導、重新詮釋莎士比亞經典角色「茱麗葉」就算諾言不曾實現  曾經愛過的回憶依舊存在     三段時空  三個為情所困的茱麗葉「帶我一起走。」那夜，秀珠鼓起勇氣對羅偉這麼說。那是1970年代的臺灣，跛腳女工秀珠成日生活在充滿油墨、紙張與機器噪音的陰暗工廠裡，對於未來、對於愛情的憧憬幾乎已消失殆盡。某日，一個年輕大學生羅偉出現，他想印一份報紙，內容是對社會改革的批判與意見。他熱切的言論和模樣，深深吸引了秀珠。那是戒嚴的年代，印刷廠老闆毫不猶豫地拒絕。但在印刷廠的後巷，秀珠偷偷叫住他，承諾私下幫他印那份報紙。看著羅偉喜出望外的笑顏，秀珠心想，不管後果有多嚴重，此刻她願意付出全世界，只為了交換那陽光世界裡的愛情幻覺……，哪怕只有一分鐘也好。 「為了你，我什麼都敢。」茱莉永遠記得小羅許下的這個承諾。30年前，歌舞團千金的她愛上布袋戲世家小羅，逢年過節廟口請戲，他兩家分占廣場兩頭表演，現實的競爭關係讓兩家父親互相仇視，茱莉和小羅只能暗暗相戀。山上的精神病院是他們唯一能夠找到的約會場所，宛若驚悚片的陰暗地點，似乎在測試他們對這段愛是否有足夠勇氣。然而戀情終被發現，震怒的茱莉父親逼迫羅家父子離開，臨走前小羅冒險去見茱莉最後一面，問她，敢不敢裝瘋進病院等他回來……？30年後，同個村落正因分手傷心的女孩阿妹，從一個外地來的男子口中知道這個故事，那名男子正要往山上精神病院而去……「我的愛，被老天爺沒收了。」在39歲最後一天，「茱麗葉」心中縈繞著這句話。歷經28次的單戀失敗後，「茱麗葉」決定在邁入40歲的前夕了結生命。騎著腳踏車入深山，他想不麻煩人、不嚇人地死掉。但就在繩子套到脖上的那一刻，突然有人出聲叫他走開。「茱麗葉」一看，山谷下十幾人的劇組正在拍廣告，劇組不但不讓他在那死，還拗他當臨演。「茱麗葉」莫名奇妙加入拍攝，休息時間聽到臨演界人人皆知的世紀愛情故事：在他們之中，有人拋棄正職跑起龍套，為的只是希望有天能在片廠與已成大明星的兒時愛慕對象相見。看到比自己對愛更執著的怪咖，「茱麗葉」突然有了活下去的勇氣，說不定，還能有第29次單戀？"])))
